### Backtesting Engine

This notebook runs a historical backtest for all previously generated portfolio selections.

**Workflow:**
1.  **Setup:** Configure all paths and backtesting parameters.
2.  **Load Price Data:** Load the master historical daily prices for all tickers.
3.  **Discover Selections:** Find all selection data/parameter file pairs from the output directory.
4.  **Execute Backtests:** Iterate through each selection file, run a T+1 to T+2 simulation for each weighting scheme (EW, IV, SW), and collect performance metrics.
5.  **Save Results:** Consolidate the results from this run with any previous backtest results, removing duplicates and keeping the latest run.
6.  **Verify:** Load the master results file to confirm the process was successful.

### Setup and Configuration

This cell defines all parameters for the backtesting run, including file paths and simulation constants.

In [1]:
import sys
from pathlib import Path
import pandas as pd
import logging
from tqdm.notebook import tqdm # For a nice progress bar

# --- Project Path Setup ---
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR
if str(ROOT_DIR) not in sys.path:
    sys.path.append(str(ROOT_DIR))
DATA_DIR = ROOT_DIR / 'data'
SRC_DIR = ROOT_DIR / 'src'
if str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))

# --- Local Imports ---
from config import DATE_STR, DAILY_RISK_FREE_RATE
import utils

# # --- Backtesting Parameters ---
# RISK_FREE_RATE_DAILY = 0.04 / 252

# These columns define a unique backtest. If we run a backtest for the same
# selection_date and scheme, the old result will be overwritten.
# We add strategy params to distinguish runs with different tuning.
UNIQUE_RUN_KEYS = [
    'selection_date', 'scheme', 'n_select_requested', 'inv_vol_col_name',
    'filter_min_price', 'filter_min_avg_volume_m', 'filter_min_roe_pct',
    'filter_max_debt_eq', 'score_weight_rsi', 'score_weight_change',
    'score_weight_rel_volume', 'score_weight_volatility'
]

# --- File Path Construction (using pathlib) ---
SELECTION_RESULTS_DIR = ROOT_DIR / 'output' / 'selection_results'
BACKTEST_OUTPUT_DIR = ROOT_DIR / 'output' / 'backtest_results'
HISTORICAL_PRICES_PATH = DATA_DIR / 'df_adj_close.parquet'
MASTER_RESULTS_PATH = BACKTEST_OUTPUT_DIR / 'backtest_master_results.parquet'
LOG_DIR = ROOT_DIR / 'logs'

# --- Notebook Setup ---
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.4f}'.format)
%load_ext autoreload
%autoreload 2

# --- Initialize Logging ---
log_filepath = utils.setup_backtest_logging(LOG_DIR)
run_timestamp = pd.Timestamp.now().strftime("%Y-%m-%d %H:%M:%S")

print(f"ROOT_DIR: {ROOT_DIR}")
print(f"Price Data Path: {HISTORICAL_PRICES_PATH}")
print(f"Selection Results Path: {SELECTION_RESULTS_DIR}")
print(f"Master Backtest Output Path: {MASTER_RESULTS_PATH}")

2025-06-21 19:25:03,823 - INFO - Logging initialized. Log file: c:\Users\ping\Files_win10\python\py311\stocks\logs\backtest_run_20250621_192503.log
ROOT_DIR: c:\Users\ping\Files_win10\python\py311\stocks
Price Data Path: c:\Users\ping\Files_win10\python\py311\stocks\data\df_adj_close.parquet
Selection Results Path: c:\Users\ping\Files_win10\python\py311\stocks\output\selection_results
Master Backtest Output Path: c:\Users\ping\Files_win10\python\py311\stocks\output\backtest_results\backtest_master_results.parquet


### Step 1: Load Historical Price Data

In [2]:
print(f"--- Step 1: Loading historical price data ---")
df_adj_close = utils.load_price_data(HISTORICAL_PRICES_PATH)

if df_adj_close is not None:
    print("Price data loaded successfully.")
    display(df_adj_close.head(3))
    display(df_adj_close.tail(3))
else:
    print("ERROR: Halting execution as price data failed to load.")

--- Step 1: Loading historical price data ---
2025-06-21 19:25:04,138 - INFO - Loading historical price data from: c:\Users\ping\Files_win10\python\py311\stocks\data\df_adj_close.parquet
2025-06-21 19:25:04,471 - INFO - Successfully loaded and prepared price data. Shape: (347, 1554)
Price data loaded successfully.


Ticker,A,AA,AAL,AAON,AAPL,ABBV,ABEV,ABNB,ABT,ACGL,ACHR,ACI,ACM,ACN,ACWI,ACWV,ACWX,ADBE,ADC,ADI,ADM,ADP,ADSK,ADT,AEE,AEG,AEM,AEP,AER,AES,AFG,AFL,AFRM,AGCO,AGG,AGI,AGNC,AIG,AIQ,AIRR,AIT,AIZ,AJG,AKAM,AL,ALB,ALC,ALGN,ALK,ALL,ALLE,ALLY,ALNY,ALSN,ALV,AM,AMAT,AMCR,AMD,AME,AMGN,AMH,AMLP,AMP,AMT,AMX,AMZN,AN,ANET,ANSS,AON,AOS,APA,APD,APG,APH,APO,APP,APPF,APTV,AR,ARCC,ARE,ARES,ARGX,ARKB,ARKK,ARM,ARMK,ARW,AS,ASML,ASND,ASR,ASTS,ASX,ATI,ATO,ATR,AU,AUR,AVAV,AVB,AVDE,AVDV,AVEM,AVGO,AVLV,AVTR,AVUS,AVUV,AVY,AWI,AWK,AXON,AXP,AXS,AXTA,AYI,AZEK,AZN,AZO,B,BA,BABA,BAC,BAH,BALL,BAM,BAP,BAX,BBAX,BBCA,BBD,BBEU,BBIN,BBIO,BBJP,BBUS,BBVA,BBWI,BBY,BCE,BCH,BCS,BDX,BEKE,BEN,BEP,BF-A,BF-B,BFAM,BG,BHP,BIDU,BIIB,BIL,BILI,BILS,BINC,BIO,BIP,BIRK,BITB,BIV,BJ,BK,BKLC,BKLN,BKNG,BKR,BLD,BLDR,BLK,BLV,BMI,BMO,BMRN,BMY,BN,BND,BNDX,BNS,BNT,BNTX,BOKF,BOND,BOXX,BP,BPMC,BPOP,BR,BRBR,BRFS,BRK-A,BRK-B,BRKR,BRO,BROS,BRX,BSAC,BSBR,BSCP,BSCQ,BSCR,BSV,BSX,BSY,BTI,BUD,BUFR,BURL,BWA,BWXT,BX,BXP,BXSL,BYD,BZ,C,CACC,CACI,CAE,CAG,CAH,CALF,CARR,CART,CASY,CAT,CAVA,CB,CBOE,CBRE,CBSH,CCCS,CCEP,CCI,CCJ,CCK,CCL,CDE,CDNS,CDP,CDW,CE,CEF,CEG,CELH,CF,CFG,CFLT,CFR,CG,CGCP,CGDV,CGGO,CGGR,CGMU,CGUS,CGXU,CHD,CHDN,CHE,CHH,CHKP,CHRD,CHRW,CHT,CHTR,CHWY,CI,CIBR,CIEN,CIGI,CINF,CL,CLH,CLS,CLX,CM,CMA,CMCSA,CME,CMF,CMG,CMI,CMS,CNA,CNC,CNH,CNI,CNM,CNP,CNQ,COF,COHR,COIN,COKE,COO,COOP,COP,COR,CORT,COST,COWZ,CP,CPAY,CPB,CPNG,CPRT,CPT,CQP,CR,CRBG,CRDO,CRH,CRK,CRL,CRM,CRS,CRWD,CSCO,CSGP,CSL,CSX,CTAS,CTRA,CTSH,CTVA,CUBE,CVE,CVLT,CVNA,CVS,CVX,CW,CWAN,CWB,CWEN,CWEN-A,CWST,CX,CYBR,D,DAL,DAR,DASH,DAY,DB,DBEF,DBX,DCI,DD,DDOG,DDS,DE,DECK,DELL,DEO,DFAC,DFAE,DFAI,DFAS,DFAT,DFAU,DFAX,DFCF,DFEM,DFIC,DFIV,DFLV,DFSD,DFSV,DFUS,DFUV,DG,DGRO,DGRW,DGX,DHI,DHR,DIA,DIHP,DINO,DIS,DIVO,DKNG,DKS,DLB,DLN,DLR,DLTR,DOC,DOCS,DOCU,DON,DOV,DOW,DOX,DPZ,DRI,DRS,DSGX,DSI,DT,DTE,DTM,DUHP,DUK,DUOL,DVA,DVN,DVY,DXCM,DXJ,DY,DYNF,E,EA,EAGG,EAT,EBAY,EBR,EC,ECL,ED,EDU,EDV,EEM,EEMV,EFA,EFAV,EFG,EFV,EFX,EG,EGP,EHC,EIX,EL,ELAN,ELF,ELS,ELV,EMA,EMB,EME,EMN,EMR,EMXC,ENB,ENPH,ENSG,ENTG,EOG,EPAM,EPD,EPI,EPRT,EQH,EQIX,EQNR,EQR,EQT,ERIC,ERIE,ERJ,ES,ESAB,ESGD,ESGE,ESGU,ESGV,ESLT,ESNT,ESS,ESTC,ET,ETN,ETR,ETSY,EUFN,EVR,EVRG,EW,EWBC,EWJ,EWT,EWY,EWZ,EXAS,EXC,EXE,EXEL,EXLS,EXP,EXPD,EXPE,EXR,EZU,F,FAF,FANG,FAST,FBCG,FBIN,FBND,FBTC,FCNCA,FCX,FDL,FDN,FDS,FDVV,FDX,FE,FELC,FELG,FER,FERG,FEZ,FFIV,FHN,FI,FICO,FIS,FITB,FIVE,FIX,FIXD,FLEX,FLOT,FLR,FLS,FLUT,FMS,FMX,FN,FND,FNDA,FNDE,FNDF,FNDX,FNF,FNV,FOUR,FOX,FOXA,FPE,FR,FRHC,FRT,FSEC,FSK,FSLR,FSS,FSV,FTAI,FTCS,FTEC,FTI,FTNT,FTS,FTSM,FTV,FUTU,FV,FVD,FWONA,FWONK,FXI,FYBR,G,GAP,GBIL,GBTC,GD,GDDY,GDX,GDXJ,GE,GEHC,GEN,GFI,GFL,GFS,GGAL,GGG,GH,GIB,GIL,GILD,GIS,GL,GLD,GLDM,GLPI,GLW,GM,GMAB,GME,GMED,GNRC,GOOG,GOOGL,GOVT,GPC,GPK,GPN,GRAB,GRMN,GS,GSIE,GSK,GSLC,GTLB,GTLS,GUNR,GVI,GWRE,GWW,H,HAL,HALO,HAS,HBAN,HCA,HD,HDB,HDV,HEFA,HEI,HEI-A,HES,HESM,HIG,HII,HIMS,HLI,HLN,HLNE,HLT,HMC,HMY,HOLX,HON,HOOD,HPE,HPQ,HQY,HRB,HRL,HSBC,HSIC,HST,HSY,HTHT,HUBB,HUBS,HUM,HWM,HYD,HYG,HYLB,IAGG,IAU,IBB,IBDR,IBIT,IBKR,IBM,IBN,ICE,ICL,ICLR,ICSH,IDA,IDEV,IDV,IDXX,IEF,IEFA,IEI,IEMG,IEUR,IEX,IFF,IGF,IGIB,IGM,IGSB,IGV,IHG,IHI,IJH,IJJ,IJK,IJR,IJS,IJT,ILMN,IMO,INCY,INDA,INFA,INFY,ING,INGR,INSM,INTC,INTU,INVH,IONQ,IONS,IOO,IOT,IP,IPG,IQLT,IQV,IR,IRM,ISRG,ISTB,IT,ITA,ITOT,ITT,ITUB,ITW,IUSB,IUSG,IUSV,IVE,IVV,IVW,IVZ,IWB,IWD,IWF,IWM,IWN,IWO,IWP,IWR,IWS,IWV,IWY,IX,IXJ,IXN,IXUS,IYF,IYR,IYW,J,JAAA,JAVA,JAZZ,JBHT,JBL,JBTM,JCI,JCPB,JD,JEF,JEPI,JEPQ,JGRO,JHMM,JHX,JIRE,JKHY,JLL,JMBS,JMST,JNJ,JNK,JNPR,JOBY,JPIE,JPM,JPST,JQUA,JXN,K,KB,KBR,KBWB,KD,KDP,KEP,KEX,KEY,KEYS,KGC,KHC,KIM,KKR,KLAC,KMB,KMI,KMX,KNG,KNSL,KNTK,KNX,KO,KR,KRC,KRE,KSPI,KT,KTOS,KVUE,KVYO,KWEB,L,LAD,LAMR,LBRDA,LBRDK,LCID,LDOS,LECO,LEGN,LEN,LEVI,LH,LHX,LI,LII,LIN,LITE,LKQ,LLY,LLYVA,LLYVK,LMBS,LMT,LNG,LNT,LNW,LOGI,LOW,LPLA,LPX,LQD,LRCX,LRN,LSCC,LTH,LULU,LUV,LVS,LW,LYB,LYFT,LYG,LYV,MA,MAA,MANH,MAR,MAS,MASI,MAT,MBB,MBLY,MCD,MCHI,MCHP,MCK,MCO,MDB,MDGL,MDLZ,MDT,MDY,MEDP,MELI,MET,META,MFC,MFG,MGA,MGC,MGK,MGM,MGV,MHK,MIDD,MINT,MKC,MKL,MKSI,MKTX,MLI,MLM,MMC,MMM,MMYT,MNDY,MNST,MO,MOAT,MOH,MORN,MOS,MP,MPC,MPLX,MPWR,MRK,MRNA,

Ticker,A,AA,AAL,AAON,AAPL,ABBV,ABEV,ABNB,ABT,ACGL,ACHR,ACI,ACM,ACN,ACWI,ACWV,ACWX,ADBE,ADC,ADI,ADM,ADP,ADSK,ADT,AEE,AEG,AEM,AEP,AER,AES,AFG,AFL,AFRM,AGCO,AGG,AGI,AGNC,AIG,AIQ,AIRR,AIT,AIZ,AJG,AKAM,AL,ALB,ALC,ALGN,ALK,ALL,ALLE,ALLY,ALNY,ALSN,ALV,AM,AMAT,AMCR,AMD,AME,AMGN,AMH,AMLP,AMP,AMT,AMX,AMZN,AN,ANET,ANSS,AON,AOS,APA,APD,APG,APH,APO,APP,APPF,APTV,AR,ARCC,ARE,ARES,ARGX,ARKB,ARKK,ARM,ARMK,ARW,AS,ASML,ASND,ASR,ASTS,ASX,ATI,ATO,ATR,AU,AUR,AVAV,AVB,AVDE,AVDV,AVEM,AVGO,AVLV,AVTR,AVUS,AVUV,AVY,AWI,AWK,AXON,AXP,AXS,AXTA,AYI,AZEK,AZN,AZO,B,BA,BABA,BAC,BAH,BALL,BAM,BAP,BAX,BBAX,BBCA,BBD,BBEU,BBIN,BBIO,BBJP,BBUS,BBVA,BBWI,BBY,BCE,BCH,BCS,BDX,BEKE,BEN,BEP,BF-A,BF-B,BFAM,BG,BHP,BIDU,BIIB,BIL,BILI,BILS,BINC,BIO,BIP,BIRK,BITB,BIV,BJ,BK,BKLC,BKLN,BKNG,BKR,BLD,BLDR,BLK,BLV,BMI,BMO,BMRN,BMY,BN,BND,BNDX,BNS,BNT,BNTX,BOKF,BOND,BOXX,BP,BPMC,BPOP,BR,BRBR,BRFS,BRK-A,BRK-B,BRKR,BRO,BROS,BRX,BSAC,BSBR,BSCP,BSCQ,BSCR,BSV,BSX,BSY,BTI,BUD,BUFR,BURL,BWA,BWXT,BX,BXP,BXSL,BYD,BZ,C,CACC,CACI,CAE,CAG,CAH,CALF,CARR,CART,CASY,CAT,CAVA,CB,CBOE,CBRE,CBSH,CCCS,CCEP,CCI,CCJ,CCK,CCL,CDE,CDNS,CDP,CDW,CE,CEF,CEG,CELH,CF,CFG,CFLT,CFR,CG,CGCP,CGDV,CGGO,CGGR,CGMU,CGUS,CGXU,CHD,CHDN,CHE,CHH,CHKP,CHRD,CHRW,CHT,CHTR,CHWY,CI,CIBR,CIEN,CIGI,CINF,CL,CLH,CLS,CLX,CM,CMA,CMCSA,CME,CMF,CMG,CMI,CMS,CNA,CNC,CNH,CNI,CNM,CNP,CNQ,COF,COHR,COIN,COKE,COO,COOP,COP,COR,CORT,COST,COWZ,CP,CPAY,CPB,CPNG,CPRT,CPT,CQP,CR,CRBG,CRDO,CRH,CRK,CRL,CRM,CRS,CRWD,CSCO,CSGP,CSL,CSX,CTAS,CTRA,CTSH,CTVA,CUBE,CVE,CVLT,CVNA,CVS,CVX,CW,CWAN,CWB,CWEN,CWEN-A,CWST,CX,CYBR,D,DAL,DAR,DASH,DAY,DB,DBEF,DBX,DCI,DD,DDOG,DDS,DE,DECK,DELL,DEO,DFAC,DFAE,DFAI,DFAS,DFAT,DFAU,DFAX,DFCF,DFEM,DFIC,DFIV,DFLV,DFSD,DFSV,DFUS,DFUV,DG,DGRO,DGRW,DGX,DHI,DHR,DIA,DIHP,DINO,DIS,DIVO,DKNG,DKS,DLB,DLN,DLR,DLTR,DOC,DOCS,DOCU,DON,DOV,DOW,DOX,DPZ,DRI,DRS,DSGX,DSI,DT,DTE,DTM,DUHP,DUK,DUOL,DVA,DVN,DVY,DXCM,DXJ,DY,DYNF,E,EA,EAGG,EAT,EBAY,EBR,EC,ECL,ED,EDU,EDV,EEM,EEMV,EFA,EFAV,EFG,EFV,EFX,EG,EGP,EHC,EIX,EL,ELAN,ELF,ELS,ELV,EMA,EMB,EME,EMN,EMR,EMXC,ENB,ENPH,ENSG,ENTG,EOG,EPAM,EPD,EPI,EPRT,EQH,EQIX,EQNR,EQR,EQT,ERIC,ERIE,ERJ,ES,ESAB,ESGD,ESGE,ESGU,ESGV,ESLT,ESNT,ESS,ESTC,ET,ETN,ETR,ETSY,EUFN,EVR,EVRG,EW,EWBC,EWJ,EWT,EWY,EWZ,EXAS,EXC,EXE,EXEL,EXLS,EXP,EXPD,EXPE,EXR,EZU,F,FAF,FANG,FAST,FBCG,FBIN,FBND,FBTC,FCNCA,FCX,FDL,FDN,FDS,FDVV,FDX,FE,FELC,FELG,FER,FERG,FEZ,FFIV,FHN,FI,FICO,FIS,FITB,FIVE,FIX,FIXD,FLEX,FLOT,FLR,FLS,FLUT,FMS,FMX,FN,FND,FNDA,FNDE,FNDF,FNDX,FNF,FNV,FOUR,FOX,FOXA,FPE,FR,FRHC,FRT,FSEC,FSK,FSLR,FSS,FSV,FTAI,FTCS,FTEC,FTI,FTNT,FTS,FTSM,FTV,FUTU,FV,FVD,FWONA,FWONK,FXI,FYBR,G,GAP,GBIL,GBTC,GD,GDDY,GDX,GDXJ,GE,GEHC,GEN,GFI,GFL,GFS,GGAL,GGG,GH,GIB,GIL,GILD,GIS,GL,GLD,GLDM,GLPI,GLW,GM,GMAB,GME,GMED,GNRC,GOOG,GOOGL,GOVT,GPC,GPK,GPN,GRAB,GRMN,GS,GSIE,GSK,GSLC,GTLB,GTLS,GUNR,GVI,GWRE,GWW,H,HAL,HALO,HAS,HBAN,HCA,HD,HDB,HDV,HEFA,HEI,HEI-A,HES,HESM,HIG,HII,HIMS,HLI,HLN,HLNE,HLT,HMC,HMY,HOLX,HON,HOOD,HPE,HPQ,HQY,HRB,HRL,HSBC,HSIC,HST,HSY,HTHT,HUBB,HUBS,HUM,HWM,HYD,HYG,HYLB,IAGG,IAU,IBB,IBDR,IBIT,IBKR,IBM,IBN,ICE,ICL,ICLR,ICSH,IDA,IDEV,IDV,IDXX,IEF,IEFA,IEI,IEMG,IEUR,IEX,IFF,IGF,IGIB,IGM,IGSB,IGV,IHG,IHI,IJH,IJJ,IJK,IJR,IJS,IJT,ILMN,IMO,INCY,INDA,INFA,INFY,ING,INGR,INSM,INTC,INTU,INVH,IONQ,IONS,IOO,IOT,IP,IPG,IQLT,IQV,IR,IRM,ISRG,ISTB,IT,ITA,ITOT,ITT,ITUB,ITW,IUSB,IUSG,IUSV,IVE,IVV,IVW,IVZ,IWB,IWD,IWF,IWM,IWN,IWO,IWP,IWR,IWS,IWV,IWY,IX,IXJ,IXN,IXUS,IYF,IYR,IYW,J,JAAA,JAVA,JAZZ,JBHT,JBL,JBTM,JCI,JCPB,JD,JEF,JEPI,JEPQ,JGRO,JHMM,JHX,JIRE,JKHY,JLL,JMBS,JMST,JNJ,JNK,JNPR,JOBY,JPIE,JPM,JPST,JQUA,JXN,K,KB,KBR,KBWB,KD,KDP,KEP,KEX,KEY,KEYS,KGC,KHC,KIM,KKR,KLAC,KMB,KMI,KMX,KNG,KNSL,KNTK,KNX,KO,KR,KRC,KRE,KSPI,KT,KTOS,KVUE,KVYO,KWEB,L,LAD,LAMR,LBRDA,LBRDK,LCID,LDOS,LECO,LEGN,LEN,LEVI,LH,LHX,LI,LII,LIN,LITE,LKQ,LLY,LLYVA,LLYVK,LMBS,LMT,LNG,LNT,LNW,LOGI,LOW,LPLA,LPX,LQD,LRCX,LRN,LSCC,LTH,LULU,LUV,LVS,LW,LYB,LYFT,LYG,LYV,MA,MAA,MANH,MAR,MAS,MASI,MAT,MBB,MBLY,MCD,MCHI,MCHP,MCK,MCO,MDB,MDGL,MDLZ,MDT,MDY,MEDP,MELI,MET,META,MFC,MFG,MGA,MGC,MGK,MGM,MGV,MHK,MIDD,MINT,MKC,MKL,MKSI,MKTX,MLI,MLM,MMC,MMM,MMYT,MNDY,MNST,MO,MOAT,MOH,MORN,MOS,MP,MPC,MPLX,MPWR,MRK,MRNA,

### Step 2: Discover Selection Files to Backtest

In [3]:
if df_adj_close is not None:
    print(f"\n--- Step 2: Discovering selection files in {SELECTION_RESULTS_DIR} ---")
    file_pairs = utils.find_and_pair_selection_files(SELECTION_RESULTS_DIR)
    
    if not file_pairs:
        print("No file pairs found to process. Halting.")
    else:
        print(f"Found {len(file_pairs)} pairs to backtest.")
        # Print the first few for inspection
        for i, (d, p) in enumerate(file_pairs[:3]):
            print(f"  Pair {i+1}: {d.name} | {p.name}")
else:
    print("Skipping file discovery.")
    file_pairs = []


--- Step 2: Discovering selection files in c:\Users\ping\Files_win10\python\py311\stocks\output\selection_results ---
2025-06-21 19:25:07,708 - INFO - Searching for selection files in: c:\Users\ping\Files_win10\python\py311\stocks\output\selection_results
2025-06-21 19:25:07,801 - INFO - Found 41 paired data and parameter files.
Found 41 pairs to backtest.
  Pair 1: 2025-04-25_short_term_mean_reversion.parquet | 2025-04-25_short_term_mean_reversion_params.json
  Pair 2: 2025-04-28_short_term_mean_reversion.parquet | 2025-04-28_short_term_mean_reversion_params.json
  Pair 3: 2025-04-29_short_term_mean_reversion.parquet | 2025-04-29_short_term_mean_reversion_params.json


In [8]:
file_pairs[-5::]

[(WindowsPath('c:/Users/ping/Files_win10/python/py311/stocks/output/selection_results/2025-06-16_short_term_mean_reversion.parquet'),
  WindowsPath('c:/Users/ping/Files_win10/python/py311/stocks/output/selection_results/2025-06-16_short_term_mean_reversion_params.json')),
 (WindowsPath('c:/Users/ping/Files_win10/python/py311/stocks/output/selection_results/2025-06-17_short_term_mean_reversion.parquet'),
  WindowsPath('c:/Users/ping/Files_win10/python/py311/stocks/output/selection_results/2025-06-17_short_term_mean_reversion_params.json')),
 (WindowsPath('c:/Users/ping/Files_win10/python/py311/stocks/output/selection_results/2025-06-18_short_term_mean_reversion.parquet'),
  WindowsPath('c:/Users/ping/Files_win10/python/py311/stocks/output/selection_results/2025-06-18_short_term_mean_reversion_params.json')),
 (WindowsPath('c:/Users/ping/Files_win10/python/py311/stocks/output/selection_results/2025-06-19_short_term_mean_reversion.parquet'),
  WindowsPath('c:/Users/ping/Files_win10/python

### Step 3: Execute Backtests

In [4]:
all_performance_records = []
if file_pairs:
    print(f"\n--- Step 3: Executing backtests for {len(file_pairs)} selection dates ---")
    
    # Use tqdm for a progress bar
    for data_file, param_file in tqdm(file_pairs, desc="Backtesting Selections"):
        records = utils.process_backtest_for_pair(
            data_file=data_file,
            param_file=param_file,
            df_adj_close=df_adj_close,
            risk_free_rate_daily=DAILY_RISK_FREE_RATE,
            run_timestamp=run_timestamp,
            log_filepath=log_filepath
        )
        all_performance_records.extend(records)
        
    print(f"Backtesting complete. Generated {len(all_performance_records)} new performance records.")
else:
    print("No backtests to execute.")


--- Step 3: Executing backtests for 41 selection dates ---


Backtesting Selections:   0%|          | 0/41 [00:00<?, ?it/s]

2025-06-21 19:25:08,074 - INFO - ------------------------------
2025-06-21 19:25:08,076 - INFO - Initiating Backtest Run...
2025-06-21 19:25:08,077 - INFO -   Date          : 2025-04-25
2025-06-21 19:25:08,078 - INFO -   Scheme        : EW
2025-06-21 19:25:08,079 - INFO -   Num Tickers   : 10
2025-06-21 19:25:08,081 - INFO -   Selection Date Used: 2025-04-25
2025-06-21 19:25:08,082 - INFO -   Buy Date           : 2025-04-28
2025-06-21 19:25:08,084 - INFO -   Sell Date          : 2025-04-29
2025-06-21 19:25:08,090 - INFO -   Trades Executed: 10/10
2025-06-21 19:25:08,091 - INFO -   Portfolio Return : 0.0085
2025-06-21 19:25:08,092 - INFO -   Win Rate         : 70.00%
2025-06-21 19:25:08,093 - INFO - Backtest simulation for 'EW' on 2025-04-25 completed.
2025-06-21 19:25:08,095 - INFO - ------------------------------
2025-06-21 19:25:08,096 - INFO - Initiating Backtest Run...
2025-06-21 19:25:08,097 - INFO -   Date          : 2025-04-25
2025-06-21 19:25:08,098 - INFO -   Scheme        : I

### Step 4: Consolidate and Save Results

In [5]:
if all_performance_records:
    print("\n--- Step 4: Consolidating and saving all results ---")
    utils.update_and_save_results(
        new_records=all_performance_records,
        results_path=MASTER_RESULTS_PATH,
        unique_key_cols=UNIQUE_RUN_KEYS
    )
else:
    print("\nNo new results to save.")



--- Step 4: Consolidating and saving all results ---
2025-06-21 19:25:13,047 - INFO - Updating results file at: c:\Users\ping\Files_win10\python\py311\stocks\output\backtest_results\backtest_master_results.parquet
2025-06-21 19:25:13,051 - INFO - Loading backtest_master_results.parquet to merge with new results.
2025-06-21 19:25:13,105 - INFO - Successfully saved 111 consolidated records to backtest_master_results.parquet


### Step 5: Verify Saved Results

In [6]:
print("\n--- Step 5: Verifying saved master results file ---")
if MASTER_RESULTS_PATH.exists():
    df_results = pd.read_parquet(MASTER_RESULTS_PATH)
    print(f"Successfully loaded master results file with {len(df_results)} records.")
    print("Displaying tail of results dataframe (latest runs):")
    display(df_results.tail())
else:
    print(f"ERROR: Master results file was not found at {MASTER_RESULTS_PATH}")


--- Step 5: Verifying saved master results file ---
Successfully loaded master results file with 111 records.
Displaying tail of results dataframe (latest runs):


,actual_selection_date_used,average_return,filter_max_debt_eq,filter_min_avg_volume_m,filter_min_price,filter_min_roe_pct,inv_vol_col_name,log_file,n_select_actual,n_select_requested,num_attempted_trades,num_failed_or_skipped_trades,num_selected_tickers,num_successful_trades,portfolio_return,portfolio_return_normalized,run_timestamp,scheme,score_weight_change,score_weight_rel_volume,score_weight_rsi,score_weight_volatility,selection_date,sharpe_ratio_period,std_dev_return,total_weight_traded,win_rate
106,2025-04-28,0.0114,1.5000,2.0000,10.0000,5.0000,ATR/Price %,backtest_run_20250621_192503.log,10,10,10,0,10,10,0.0105,0.0105,2025-06-21 19:25:03,IV,0.3500,0.2000,0.3500,0.1000,2025-04-28,0.5434,0.0208,1.0000,0.8000
107,2025-04-28,0.0114,1.5000,2.0000,10.0000,5.0000,ATR/Price %,backtest_run_20250621_192503.log,10,10,10,0,10,10,0.0128,0.0128,2025-06-21 19:25:03,SW,0.3500,0.2000,0.3500,0.1000,2025-04-28,0.5434,0.0208,1.0000,0.8000
108,2025-04-25,0.0085,1.5000,2.0000,10.0000,5.0000,ATR/Price %,backtest_run_20250621_192503.log,10,10,10,0,10,10,0.0085,0.0085,2025-06-21 19:25:03,EW,0.3500,0.2000,0.3500,0.1000,2025-04-25,0.4129,0.0203,1.0000,0.7000
109,2025-04-25,0.0085,1.5000,2.0000,10.0000,5.0000,ATR/Price %,backtest_run_20250621_192503.log,10,10,10,0,10,10,0.0075,0.0075,2025-06-21 19:25:03,IV,0.3500,0.2000,0.3500,0.1000,2025-04-25,0.4129,0.0203,1.0000,0.7000
110,2025-04-25,0.0085,1.5000,2.0000,10.0000,5.0000,ATR/Price %,backtest_run_20250621_192503.log,10,10,10,0,10,10,0.0096,0.0096,2025-06-21 19:25:03,SW,0.3500,0.2000,0.3500,0.1000,2025-04-25,0.4129,0.0203,1.0000,0.7000
